In [1]:
# move to the project root folder
%cd ../../

c:\Users\sanatbyeka\Desktop\calcium_scoring


In [2]:
import numpy as np

# Bounding box center in world coordinates [X, Y, Z]
bbox_center_world = np.array([35.287925720214844, 7.3646368980407715, 1867.57958984375])

# Voxel spacing along each axis [X, Y, Z]
spacing = np.array([0.37599999, 0.37599999, 3.0])

# Image size [Width, Height, Slices]
image_size = np.array([512, 512, 47])

# Image origin [X, Y, Z]
image_origin = np.array([93.57398224, -46.68701935, 1761.75])

# Convert world coordinates to voxel coordinates
bbox_center_voxel = bbox_center_world / spacing

# Subtract the image origin to account for the offset
bbox_center_voxel -= image_origin / spacing

# Convert voxel coordinates to pixel coordinates
# Ensure that the values are within the image dimensions
bbox_center_pixel = np.clip(bbox_center_voxel[:2], 0, image_size[:2] - 1)

# The 'clip' function ensures that the pixel coordinates do not exceed the image size.
# bbox_center_pixel will contain the bounding box center in pixel coordinates.

print("Bounding Box Center in Pixel Coordinates:", bbox_center_pixel)


Bounding Box Center in Pixel Coordinates: [  0.         143.75440874]


In [2]:
import os
import sys
from pathlib import Path 
PROJECT_PATH = os.getcwd()

assert PROJECT_PATH.split('\\')[-1] == 'calcium_scoring', print("Not in the calcium_scoring project root folder path")

In [3]:
src_dir = os.path.join(os.getcwd(), 'src')
sys.path.append(src_dir)
from src.visualizations import viz
#viz.plot_imgs_bboxes(r'C:\Users\sanatbyeka\Desktop\calcium_scoring\data\datasets\aorta_dataset\images\train', r'C:\Users\sanatbyeka\Desktop\calcium_scoring\data\datasets\aorta_dataset\labels\train')

In [4]:
# create yaml file
yaml_content = f""" 
    path: aorta_dataset
    train: images/train
    val: images/val
    test: images/test

    names:
        0: aorta
"""

with Path('data/datasets/aorta_dataset/data.yaml').open('w') as f:
    f.write(yaml_content)

In [7]:
%pip install -q "openvino-dev>=2023.0.0" "nncf>=2.5.0"
%pip install -q "ultralytics==8.0.171" onnx

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics YOLOv8.0.171  Python-3.10.12 torch-2.0.1 CPU (Intel Core(TM) i7-8700 3.20GHz)
Setup complete  (12 CPUs, 15.8 GB RAM, 173.1/225.9 GB disk)


In [6]:
from ultralytics import settings
settings['datasets_dir'] = os.path.join(PROJECT_PATH, 'data\\datasets')
os.makedirs(os.path.join(PROJECT_PATH, 'experiments\\runs'), exist_ok=True)
os.makedirs(os.path.join(PROJECT_PATH, 'experiments\\weights'), exist_ok=True)
settings['runs_dir'] =  os.path.join(PROJECT_PATH, 'experiments\\runs')
settings['weights_dir'] =  os.path.join(PROJECT_PATH, 'experiments\\weights')
print(settings)

{'settings_version': '0.0.2', 'datasets_dir': 'c:\\Users\\sanatbyeka\\Desktop\\calcium_scoring\\data\\datasets', 'weights_dir': 'c:\\Users\\sanatbyeka\\Desktop\\calcium_scoring\\experiments\\weights', 'runs_dir': 'c:\\Users\\sanatbyeka\\Desktop\\calcium_scoring\\experiments\\runs', 'uuid': '7d30500fe42cd01dd19c778974587b169c59f5f12014fd3e5ff5e0f7e76e53e6', 'sync': True, 'api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True}


In [8]:
save_dir = 'logs'
# os.makedirs(save_dir, exist_ok=True)

# Load a model
model = YOLO('models//yolov8n.pt')  
PROJECT = 'logs//aorta_detection' 

# training
result = model.train(
   data = 'data/datasets/aorta_dataset/data.yaml',
   task = 'detect',
   epochs = 3,
   verbose = True,
   batch = 32,
   imgsz = 640,
   patience = 50,
   save = True,
   #device = 0,
   project = PROJECT,
   cos_lr = True,
   lr0 = 0.0001,
   lrf = 0.00001,
   warmup_epochs = 3,
   warmup_bias_lr = 0.000001,
   optimizer = 'Adam',
   seed = 42,
   save_dir=save_dir
)

Ultralytics YOLOv8.0.171  Python-3.10.12 torch-2.0.1 CPU (Intel Core(TM) i7-8700 3.20GHz)
engine\trainer: task=detect, mode=train, model=models//yolov8n.pt, data=data/datasets/aorta_dataset/data.yaml, epochs=3, patience=50, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=logs//aorta_detection, name=None, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, f

RuntimeError: Dataset 'data/datasets/aorta_dataset/data.yaml' error  
Dataset 'data/datasets/aorta_dataset/data.yaml' images not found , missing path 'C:\Users\sanatbyeka\Desktop\calcium_scoring\datasets\aorta_dataset\images\val'
Note dataset download directory is 'C:\Users\sanatbyeka\Desktop\calcium_scoring\datasets'. You can update this in 'C:\Users\sanatbyeka\AppData\Roaming\Ultralytics\settings.yaml'

In [ ]:
model = YOLO('aorta_detection’/experiment_yolov8/weights/best.pt')
model.val(split='test', batch=48, imgsz=640, verbose=True, conf = 0.1, iou = 0.5)

In [ ]:
# Alternatively, we can use Python SDK to validate the model performance.


model = YOLO("./runs/detect/train/weights/best.pt")

# It'll use the data yaml file in model.pt if you don't set data.
model.val(conf=0.5)
# or you can set the data you want to val
model.val(data="./datasets/boat.yaml", conf=0.5)
 